In [1]:
from explainers.minimax_explainer import MiniMaxExplainer

## Example search tree:

In [2]:
class MiniMaxNode:
    def __init__(self, id, *, score=None, maximizing_player_turn=True, children=None, score_child=None):
        
        self.id = id

        if score:
            self.score = score
        elif score_child:
            self.score_child = score_child
            self.score = score_child.score
        else:
            raise ValueError("Provide score or score_child.")

        self.children = children or []
        self.parent = None
        self.maximizing_player_turn = maximizing_player_turn
        
        self.is_leaf = True
        if len(self.children) > 0:
            self.is_leaf = False
            for child in children:
                child.parent=self
                child.maximizing_player_turn = not self.maximizing_player_turn
    
    def __str__(self):
        return self.id

# Create a simple game tree
leaf11 = MiniMaxNode('leaf11', score=3)
leaf12 = MiniMaxNode('leaf12', score=4)
leaf21 = MiniMaxNode('leaf21', score=8)
leaf22 = MiniMaxNode('leaf22', score=2)
leaf31 = MiniMaxNode('leaf21', score=1)
leaf32 = MiniMaxNode('leaf22', score=1)

child1 = MiniMaxNode('child1', children=[leaf11, leaf12], score_child=leaf11)
child2 = MiniMaxNode('child2', children=[leaf21, leaf22], score_child=leaf22)
child3 = MiniMaxNode('child3', children=[leaf31, leaf32], score_child=leaf31)

root = MiniMaxNode('root', maximizing_player_turn=True, children=[child1, child2, child3], score_child=child1)

In [3]:
explainer = MiniMaxExplainer()

## Framework definition

In [4]:
print(explainer['lowlevel'])

Propositions:
node is leaf
node has score = ?
node is opponent player turn
node has backpropagating child = ?
node1 is better than node2
node has siblings = ?
node is best
node is worst


Implications:
node has leaf = ? ←
 (assumption) Definition of "leaf" is node.is_leaf
node has score = ? ←
 node is leaf ←
 (assumption) Definition of "leaf" is node.is_leaf
∧ (assumption) Leaf nodes have scores from the evaluation function
|| node is ¬(leaf) ←
 (assumption) Definition of "leaf" is node.is_leaf
∧ (assumption) Internal nodes have scores from children
∧ node has backpropagating child = ? ←
 node is opponent player turn ←
 (assumption) Definition of "opponent player turn" is not node.maximizing_player_turn
∧ (assumption) We assume the opponent will do their best move.
∧ node has worst = ? ←
 (assumption) By definition a node is "worst" if it's not "better" than all "siblings"
∧ Node not better than all nodes in siblings
|| node is ¬(opponent player turn) ←
 (assumption) Definition of "opp

In [5]:
print(explainer['highlevel'])

Propositions:
move is final move
move has score = ?
move is opponent player turn
move has next possible move = ?
move1 is better than move2
move has possible alternative moves = ?
move is the best
move is the best the opponent can do


Implications:
move has final move = ? ←
 (assumption) Definition of "final move" is node.is_leaf
move has score = ? ←
 move is final move ←
 (assumption) Definition of "final move" is node.is_leaf
∧ (assumption) final moves are evaluated only looking at the final position
|| move is ¬(final move) ←
 (assumption) Definition of "final move" is node.is_leaf
∧ move has next possible move = ? ←
 move is opponent player turn ←
 (assumption) Definition of "opponent player turn" is not node.maximizing_player_turn
∧ (assumption) we assume the opponent will do their best move
∧ move has the best the opponent can do = ? ←
 (assumption) By definition a move is "the best the opponent can do" if it's not "better" than all "possible alternative moves"
∧ Node not better

## Set your explanations settings

In [6]:
settings = {
            'with_framework': 'highlevel',
            'explanation_depth': 5 ,
            'assumptions_verbosity': 'verbose'
        }

explainer.configure_settings(settings)

## You can now explain nodes' properties

In [7]:
explainer.explain(child1, "the best")

child1 is the best ←
 	(assumption) By definition a move is "the best" if it's "better" than all "possible alternative moves"
	∧ child1 has possible alternative moves = child2 (only showing relevant 1) ←
	 		(assumption) Definition of "possible alternative moves" is node.parent.children excluding node
	∧ child1 is better than child2 ←
	 	it leads to a better position ←
	 		(assumption) By definition, move1 is "better" than move2 if move1 score > move2 score
			∧ child1 is ¬(final move) ←
			 				(assumption) Definition of "final move" is node.is_leaf
			∧ child1 has next possible move = leaf11 ←
			 				(assumption) we assume the opponent will do their best move
							∧ leaf11 is the best the opponent can do ←
							 					(assumption) By definition a move is "the best the opponent can do" if it's not "better" than all "possible alternative moves"
												∧ leaf11 has possible alternative moves = leaf12 (only showing relevant 1) ∧ leaf11 is ¬(better than leaf12)
			∧ child2 is ¬(f

In [8]:
high_abstraction_settings = {
            'with_framework': 'highlevel',
            'explanation_depth': 4 ,
            'print_implicit_assumptions': False,
            'assumptions_verbosity': 'if_asked',
            'print_mode': 'verbal'
        }

explainer.configure_settings(high_abstraction_settings)

In [10]:
explainer.explain(child1, "the best", explanation_depth=2)

child1 is the best (because
 	child1 has as possible alternative moves child2 (only showing relevant 1) and child1 is better than child2)


In [11]:
explainer.explain(child1, "the best")

child1 is the best (because
 	child1 has as possible alternative moves child2 (only showing relevant 1)
	and child1 is better than child2 (because
	 	it leads to a better position (because
	 		child1 has as next possible move leaf11 (because
			 				leaf11 is the best the opponent can do)
			and child2 has as next possible move leaf22 (because
			 				leaf22 is the best the opponent can do))))


In [12]:
settings = {
            'with_framework': 'lowlevel',
            'explanation_depth': 2 ,
            'assumptions_verbosity' : 'if_asked',
            'print_mode': 'logic'
        }

explainer.configure_settings(settings)

In [13]:
explainer.explain(child1, "worst", explanation_depth = 0)

child1 is ¬(worst)


In [14]:
explainer.explain(child1, "worst", explanation_depth = 1)

child1 is ¬(worst) ←
 	child1 has siblings = child2, child3
	∧ child1 is better than child2 ∧ child1 is better than child3


In [15]:
explainer.explain(child1, "worst")

child1 is ¬(worst) ←
 	child1 has siblings = child2, child3
	∧ child1 is better than child2 ←
	 		child1 has score = 3 ∧ child2 has score = 2
	∧ child1 is better than child3 ←
	 		child1 has score = 3 ∧ child3 has score = 1


In [16]:
explainer.explain(child1, "worst", explanation_depth = 3, print_depth=True)

child1 is ¬(worst) ←
 	Depth 1:
	child1 has siblings = child2, child3
	∧ child1 is better than child2 ←
	 		Depth 2:
			child1 has score = 3 ←
			 			Depth 3:
						child1 is ¬(leaf) ∧ child1 has backpropagating child = leaf11
			∧ child2 has score = 2 ←
			 			Depth 3:
						child2 is ¬(leaf) ∧ child2 has backpropagating child = leaf22
	∧ child1 is better than child3 ←
	 		Depth 2:
			child1 has score = 3 ←
			 			Depth 3:
						child1 is ¬(leaf) ∧ child1 has backpropagating child = leaf11
			∧ child3 has score = 1 ←
			 			Depth 3:
						child3 is ¬(leaf) ∧ child3 has backpropagating child = leaf21


In [17]:
low_abstraction_settings = {
            'with_framework': 'lowlevel',
            'explanation_depth': 3 ,
            'print_implicit_assumptions': True,
            'assumptions_verbosity': 'verbose',
            'print_mode': 'logic'
        }

explainer.configure_settings(low_abstraction_settings)

In [18]:
explainer.explain(child1, "better", child2)

child1 is better than child2 ←
 	(assumption) By definition, node1 is "better" than node2 if node1 score > node2 score
	∧ child1 has score = 3 ←
	 		child1 is ¬(leaf) ←
			 			(assumption) Definition of "leaf" is node.is_leaf
			∧ (assumption) Internal nodes have scores from children
			∧ child1 has backpropagating child = leaf11 ←
			 			child1 is opponent player turn
						∧ (assumption) We assume the opponent will do their best move.
						∧ leaf11 is worst
	∧ child2 has score = 2 ←
	 		child2 is ¬(leaf) ←
			 			(assumption) Definition of "leaf" is node.is_leaf
			∧ (assumption) Internal nodes have scores from children
			∧ child2 has backpropagating child = leaf22 ←
			 			child2 is opponent player turn
						∧ (assumption) We assume the opponent will do their best move.
						∧ leaf22 is worst


In [19]:
explainer.explain(child1, "siblings")

child1 has siblings = child2, child3 ←
 	(assumption) Definition of "siblings" is node.parent.children excluding node


In [20]:
explainer.explain(child1, "best")

child1 is best ←
 	(assumption) By definition a node is "best" if it's "better" than all "siblings"
	∧ child1 has siblings = child2, child3 ←
	 		(assumption) Definition of "siblings" is node.parent.children excluding node
	∧ child1 is better than child2 ←
	 		(assumption) By definition, node1 is "better" than node2 if node1 score > node2 score
			∧ child1 has score = 3 ←
			 			child1 is ¬(leaf)
						∧ (assumption) Internal nodes have scores from children
						∧ child1 has backpropagating child = leaf11
			∧ child2 has score = 2 ←
			 			child2 is ¬(leaf)
						∧ (assumption) Internal nodes have scores from children
						∧ child2 has backpropagating child = leaf22
	∧ child1 is better than child3 ←
	 		(assumption) By definition, node1 is "better" than node2 if node1 score > node2 score
			∧ child1 has score = 3 ←
			 			child1 is ¬(leaf)
						∧ (assumption) Internal nodes have scores from children
						∧ child1 has backpropagating child = leaf11
			∧ child3 has score = 1 ←
			 		

In [21]:
explainer.explain(root, "backpropagating child")

root has backpropagating child = child1 ←
 	root is ¬(opponent player turn) ←
	 		(assumption) Definition of "opponent player turn" is not node.maximizing_player_turn
	∧ (assumption) On our turn we take the maximum rated move.
	∧ child1 is best ←
	 		(assumption) By definition a node is "best" if it's "better" than all "siblings"
			∧ child1 has siblings = child2, child3 ←
			 			(assumption) Definition of "siblings" is node.parent.children excluding node
			∧ child1 is better than child2 ←
			 			(assumption) By definition, node1 is "better" than node2 if node1 score > node2 score
						∧ child1 has score = 3 ∧ child2 has score = 2
			∧ child1 is better than child3 ←
			 			(assumption) By definition, node1 is "better" than node2 if node1 score > node2 score
						∧ child1 has score = 3 ∧ child3 has score = 1


In [22]:
explainer.explain(child1, "best", explanation_depth=6, print_depth=True)

child1 is best ←
 	Depth 1:
	(assumption) By definition a node is "best" if it's "better" than all "siblings"
	∧ child1 has siblings = child2, child3 ←
	 		Depth 2:
			(assumption) Definition of "siblings" is node.parent.children excluding node
	∧ child1 is better than child2 ←
	 		Depth 2:
			(assumption) By definition, node1 is "better" than node2 if node1 score > node2 score
			∧ child1 has score = 3 ←
			 			Depth 3:
						child1 is ¬(leaf) ←
						 				Depth 4:
										(assumption) Definition of "leaf" is node.is_leaf
						∧ (assumption) Internal nodes have scores from children
						∧ child1 has backpropagating child = leaf11 ←
						 				Depth 4:
										child1 is opponent player turn ←
										 					Depth 5:
															(assumption) Definition of "opponent player turn" is not node.maximizing_player_turn
										∧ (assumption) We assume the opponent will do their best move.
										∧ leaf11 is worst ←
										 					Depth 5:
															(assumption) By definiti

In [23]:
explainer.explain(root, "score", explanation_depth = 15, print_depth=True)

root has score = 3 ←
 	Depth 1:
	root is ¬(leaf) ←
	 		Depth 2:
			(assumption) Definition of "leaf" is node.is_leaf
	∧ (assumption) Internal nodes have scores from children
	∧ root has backpropagating child = child1 ←
	 		Depth 2:
			root is ¬(opponent player turn) ←
			 			Depth 3:
						(assumption) Definition of "opponent player turn" is not node.maximizing_player_turn
			∧ (assumption) On our turn we take the maximum rated move.
			∧ child1 is best ←
			 			Depth 3:
						(assumption) By definition a node is "best" if it's "better" than all "siblings"
						∧ child1 has siblings = child2, child3 ←
						 				Depth 4:
										(assumption) Definition of "siblings" is node.parent.children excluding node
						∧ child1 is better than child2 ←
						 				Depth 4:
										(assumption) By definition, node1 is "better" than node2 if node1 score > node2 score
										∧ child1 has score = 3 ←
										 					Depth 5:
															child1 is ¬(leaf) ←
															 						Depth 6:
		

Track down that Not before the "Considering your definition of leaf"

In [24]:
explainer.query_explanation(root, "Why is child 1 maxoptimal?")